In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math
from concurrent.futures import ThreadPoolExecutor

# Encontrar todas as marcas
url = 'https://www.drogariasminasmais.com.br/medicamentos'
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

marcas = soup.find_all("label", class_="vtex-checkbox__label w-100 c-on-base pointer")
marcas_corr = []
for marca in marcas:
    marcas_corr.append(str(marca).split('>')[1].split('<')[0])
for i in range(len(marcas_corr)):
    marcas_corr[i] = marcas_corr[i].lower().replace(" ", '-')
marcas_corr = marcas_corr[19:]

print(marcas_corr)

['hypera', 'ache-rx', 'eurofarma-rx', 'ems-generico', 'sigma-pharma-rx', 'cimed', 'germed-generico', 'libbs', 'apsen', 'biolab-rx', 'supera', 'medley-genericos', 'momenta', 'farmoquimica', 'eurofarma-generico', 'myralis', 'abbott-rx', 'teuto-generico', 'ache-otc', 'sanofi-rx', 'cristalia/latinofarma', 'sanofi-otc', 'prati-donaduzzi-generico', 'torrent', 'legrand-generico', 'biolab-generico', 'uniao-quimica-similar', 'uniao-quimica-genom', 'neo-quimica-generico', 'pharlab', 'neo-quimica-similar', 'nova-quimica-generico', 'geolab-generico', 'gsk-otc', 'novartis-rx', 'marjan', 'zanphy', 'gsk-rx', 'uniao-quimica-generico', 'procter--gambl', 'biolab-avert-otc', 'alcon', 'merck-rx', 'biosintetica-generico', 'servier', 'allergan', 'teuto-similar', 'ems-similar', 'bayer-rx', 'globo-generico', 'cellera', 'astrazeneca', 'sandoz-generico', 'herbarium', 'divcom', 'airela', 'zodiac-adium', 'sankyo-pharma', 'ranbaxy', 'natulab', 'bayer-otc', 'takeda', 'legrand-rx', 'boehringer', 'sigma-pharma-otc', 

In [16]:
def extrair_valores_por_chave(string, chave, categorico):
    valores = []  # Lista para armazenar os valores encontrados
    
    # Encontrar a posição da primeira ocorrência da chave
    indice_chave = string.find(f'"{chave}":')
    
    if categorico:
        # Enquanto houver ocorrências da chave
        while indice_chave != -1:
            # Encontrar o índice do próximo caractere após a ocorrência da chave
            indice_inicio_valor = indice_chave + len(f'"{chave}":') + 1
            
            # Encontrar o índice do próximo caractere que indica o final do valor
            indice_fim_valor = string.find('"', indice_inicio_valor)
            
            # Pegar o valor entre os índices de início e fim
            valor = string[indice_inicio_valor:indice_fim_valor]
            valores.append(valor)
            
            # Encontrar a próxima ocorrência da chave após a posição atual
            indice_chave = string.find(f'"{chave}":', indice_fim_valor)
        
    
    else:
        while indice_chave != -1:
                # Encontrar o índice do próximo caractere após a ocorrência de '"Price":'
                indice_inicio_valor = indice_chave + len(f'"{chave}":')
                
                # Encontrar o índice do próximo caractere que indica o final do preço
                indice_fim_valor = indice_inicio_valor
                while string[indice_fim_valor].isdigit() or string[indice_fim_valor] == '.':
                    indice_fim_valor += 1
                
                # Pegar o valor entre os índices de início e fim
                valor = string[indice_inicio_valor:indice_fim_valor]
                valores.append(float(valor))
                
                # Encontrar a próxima ocorrência de '"Price":' após a posição atual
                indice_chave = string.find(f'"{chave}":', indice_fim_valor)
            
    return valores

# Exemplo de uso
texto = '... "productName":"Produto A", "productName":"Produto B", ...'
chave = "productName"
print(extrair_valores_por_chave(texto, chave, True))  # Saída: ['Produto A', 'Produto B']

texto = '... "productReference":"123", "productReference":"45689", ...'
chave = "productReference"
print(extrair_valores_por_chave(texto, chave, True))  # Saída: ['123', '45689']

texto = '... "Price":10.5, "Price":20.75, ...'
print(extrair_valores_por_chave(texto,"Price", False))  # Saída: [10.5, 20.75]

texto = '... "ean":"1234567890123", "ean":"9876543210987", ...'
print(extrair_valores_por_chave(texto,"ean",True))  # Saída: ['1234567890123', '9876543210987']


['Produto A', 'Produto B']
['123', '45689']
[10.5, 20.75]
['1234567890123', '9876543210987']


In [17]:
def extrair_quantidade(string):
    chave  = "recordsFiltered"

    # Encontrar a posição da primeira ocorrência da chave
    indice_chave = string.find(f'"{chave}":')

    # Encontrar o índice do próximo caractere após a ocorrência de '"Price":'
    indice_inicio_valor = indice_chave + len(f'"{chave}":')
    
    # Encontrar o índice do próximo caractere que indica o final do preço
    indice_fim_valor = indice_inicio_valor
    while string[indice_fim_valor].isdigit() or string[indice_fim_valor] == '.':
        indice_fim_valor += 1
    
    # Pegar o valor entre os índices de início e fim
    quantidade = int(string[indice_inicio_valor:indice_fim_valor])
                    
    return quantidade


texto = '... "recordsFiltered":366, "ean":"9876543210987", ...'
print(extrair_quantidade(texto))  # Saída: ['1234567890123', '9876543210987']

366


In [18]:
colunas = ['Nome', 'Preco', 'Preco Sem Desconto', 'EAN', 'SKU', 'Marca']

# Criar DataFrame vazio com as colunas especificadas
df = pd.DataFrame(columns=colunas)

# Mostrar DataFrame vazio
print(df)

Empty DataFrame
Columns: [Nome, Preco, Preco Sem Desconto, EAN, SKU, Marca]
Index: []


In [19]:
#criação das listas dos dados
ean_lista = []
sku_lista = []
nome_lista = []
preco_lista = []
precosemdesconto_lista = []
marca_lista = []


for marca in marcas_corr:

    #primeira requisicao para conseguir quantidade de produtos por marcas

    url = 'https://www.drogariasminasmais.com.br/medicamentos/'+marca+'?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1'
    print(url)

    # fazendo a requisição com o site 
    requisicao = requests.get(url)

    quantidade = extrair_quantidade(requisicao.text)
    print(quantidade)

    marca_lista.extend([marca] * quantidade)



    pagina = 1

    while pagina <= math.ceil(quantidade/15):

        url = 'https://www.drogariasminasmais.com.br/medicamentos/'+marca+'?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=' + str(pagina)

        # fazendo a requisição com o site 
        requisicao = requests.get(url)


        ean_lista.extend( extrair_valores_por_chave(requisicao.text,"ean",True))
        sku_lista.extend( extrair_valores_por_chave(requisicao.text, "productReference",True))
        nome_lista.extend( extrair_valores_por_chave(requisicao.text,"productName",True))
        preco_lista.extend( extrair_valores_por_chave(requisicao.text,'Price',False))
        precosemdesconto_lista.extend( extrair_valores_por_chave(requisicao.text,'ListPrice',False))
    # categoria_lista = extrair_valores_por_chave(requisicao.text,"originalName",True)
        
        
        pagina += 1

https://www.drogariasminasmais.com.br/medicamentos/hypera?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1
368
https://www.drogariasminasmais.com.br/medicamentos/ache-rx?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1
288
https://www.drogariasminasmais.com.br/medicamentos/eurofarma-rx?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1
234
https://www.drogariasminasmais.com.br/medicamentos/ems-generico?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1
230
https://www.drogariasminasmais.com.br/medicamentos/sigma-pharma-rx?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1
210
https://www.drogariasminasmais.com.br/medicamentos/cimed?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1
194
https://www.drogariasminasmais.com.br/medicamentos/germed-generico?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1
150
https://www.drogariasminasmais.com.br/medicamentos/libbs?initialMap

In [26]:
df['Nome'] = nome_lista
df['Preco'] = preco_lista
df['Preco Sem Desconto'] = precosemdesconto_lista
df['EAN'] = ean_lista
df['SKU'] = sku_lista
df['Marca'] = marca_lista

df.to_csv('Drogaria_Minas_Mais.csv', index = False)

In [25]:
df.to_excel('Drogaria_Minas_Mais_Excel.xlsx')

o que adicionar:

botar as colunas no dataframe
a marca

In [5]:
# Função para processar uma página
def processar_pagina(marca, pagina):
    url = 'https://www.drogariasminasmais.com.br/medicamentos/'+marca+'?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=' + str(pagina)
    requisicao = requests.get(url)
    ean_lista = extrair_valores_por_chave(requisicao.text, "ean", True)
    sku_lista = extrair_valores_por_chave(requisicao.text, "productReference", True)
    nome_lista = extrair_valores_por_chave(requisicao.text, "productName", True)
    preco_lista = extrair_valores_por_chave(requisicao.text, 'Price', False)
    precosemdesconto_lista = extrair_valores_por_chave(requisicao.text, 'ListPrice', False)
    return ean_lista, sku_lista, nome_lista, preco_lista, precosemdesconto_lista

In [6]:
ean_lista = []
sku_lista = []
nome_lista = []
preco_lista = []
precosemdesconto_lista = []
marca_lista = []

In [7]:
executor = ThreadPoolExecutor(max_workers=5)  # Por exemplo, 5 threads


for marca in marcas_corr:

    url = 'https://www.drogariasminasmais.com.br/medicamentos/'+marca+'?initialMap=c&initialQuery=medicamentos&map=category-1,brand&page=1'

    # fazendo a requisição com o site 
    requisicao = requests.get(url)

    quantidade = extrair_quantidade(requisicao.text)

    marca_lista.extend([marca] * quantidade)
    
    print("antes thread")

    pagina = 1

    while pagina <= math.ceil(quantidade/15):        
        
        # Envia uma tarefa para execução e obtém um objeto Future em troca
        resultado = executor.submit(processar_pagina(marca,pagina), 10)

        print(resultado.result())

        print('antes for')
        for ean, sku, nome, preco, precosemdesconto in resultado.result():
                print(nome)
                ean_lista.extend(ean)
                sku_lista.extend(sku)
                nome_lista.extend(nome)
                preco_lista.extend(preco)
                precosemdesconto_lista.extend(precosemdesconto)

        pagina += 1


executor.shutdown()

antes thread


TypeError: 'tuple' object is not callable

In [ ]:
df['Nome'] = nome_lista
df['Preco'] = preco_lista
df['Preco Sem Desconto'] = precosemdesconto_lista
df['EAN'] = ean_lista
df['SKU'] = sku_lista
df['Marca'] = marca_lista

df